In [4]:
import json
import os
from datetime import datetime

import tqdm
from PIL import Image

import src.data.Dataset as dt
import src.visualization.visualization_utils as vu

In [5]:
required_paths = ["/ai4eutils", "/CameraTraps", "/yolov5"]
python_path = os.environ.get("PYTHONPATH", "")
root_path = os.getcwd()

for path in required_paths:
    if not any(p.endswith(path) for p in python_path.split(":")):
        python_path += f":{root_path}/data/external{path}"

os.environ["PYTHONPATH"] = python_path

!echo "PYTHONPATH: $PYTHONPATH"

PYTHONPATH: :/Users/carlos/WORKSPACE/MegaClassifier/data/external/ai4eutils:/Users/carlos/WORKSPACE/MegaClassifier/data/external/CameraTraps:/Users/carlos/WORKSPACE/MegaClassifier/data/external/yolov5


In [6]:
DATASET_PATH = os.path.abspath("./dataset/datasetFiltered")
MASKED_PATH = os.path.abspath("./resources/masked")
DETECTION_JSON = os.path.abspath(
    "./resources/json/md_v5a/29618_images_0_003_threshold.json"
)

with open(DETECTION_JSON, "r") as f:
    result = json.load(f)

MASKED_PATH = os.path.join(
    MASKED_PATH,
    os.path.basename(DATASET_PATH),
    result["info"]["detector"].split(".")[0],
)


print()
print(f"DATASET_PATH:   {DATASET_PATH}")
print(f"MASKED_PATH:      {MASKED_PATH}")
print(f"DETECTION_JSON: {DETECTION_JSON}")


DATASET_PATH:   /Users/carlos/WORKSPACE/MegaClassifier/dataset/datasetFiltered
MASKED_PATH:      /Users/carlos/WORKSPACE/MegaClassifier/resources/masked/datasetFiltered/md_v5a
DETECTION_JSON: /Users/carlos/WORKSPACE/MegaClassifier/resources/json/md_v5a/29618_images_0_003_threshold.json


In [7]:
for image in tqdm.tqdm(result["images"]):
    input_file = os.path.join(DATASET_PATH, image["file"])
    output_file = os.path.join(MASKED_PATH, image["file"])
    detections = image["detections"]

    vu.apply_mask_on_file(input_file, output_file, detections)


100%|██████████| 29618/29618 [06:26<00:00, 76.65it/s]  


In [ ]:
from PIL import Image
import os

# Ruta base
base_path = MASKED_PATH

# Contadores para las imágenes que se pueden y no se pueden abrir
count_openable = 0
count_not_openable = 0

# Lista para almacenar las rutas de las imágenes que no se pueden abrir
not_openable_files = []

# Iterar sobre las rutas de las imágenes en el dataset
for image in result["images"]:
    file_path = image["file"]
    absolute_path = base_path + "/" + file_path
    try:
        # Intentar abrir la imagen
        img = Image.open(absolute_path)
        img.verify()  # Verificar que la imagen se puede abrir
        count_openable += 1
    except (IOError, SyntaxError) as e:
        # Si hay un error, incrementar el contador de imágenes no abiertas
        count_not_openable += 1
        not_openable_files.append(absolute_path)

# Imprimir los resultados
print(f"Number of openable images: {count_openable}")
print(f"Number of not openable images: {count_not_openable}")
print(f"Not openable files: {not_openable_files}")

Number of openable images: 29618
Number of not openable images: 0
Not openable files: []
